In [27]:
import json
import os
import re
from openai import AzureOpenAI

ENDPOINT="https://dhp-search-east-npe-0.openai.azure.com/"
MODEL="gpt-4.1"
API_VERSION="2024-02-01"
AZURE_OPEN_AI_API_KEY="2b91e7e1eb07487da18661da47fa1566"

openai_client = AzureOpenAI(
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT,
    api_key=AZURE_OPEN_AI_API_KEY
)

def write_file(m_code, path):
    
    match = re.search(r"```(?:\w+)?\n(.*?)```", m_code, re.DOTALL)
    if not match:
        raise ValueError("No code block foud")
    
    code = match.group(1).strip()
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        f.write(code)
        print("Writed to", path)

def read_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def write_json_str(json_string, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        f.write(json_string)

def load_prompt_template(path: str) -> str:
    with open(path, "r", encoding="utf-8") as file:
        return file.read()
    
def call_llm(user_instruction: str, system_instruction: str) -> str:
    conversation = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": user_instruction}
    ]

    response = openai_client.chat.completions.create(
                    model=MODEL,
                    messages=conversation,
                    timeout=600,
                    temperature=0.0)
    
    return response.choices[0].message.content

# Work flow

Step 1 - Gen Source : Plan Project -> List[Page] -> Generate each Page

Step 2 - Copy base project -> Write to file to the base project

Step 3 - Zip the project

Step 4 - Upload to Azure File Share

Step 5 - Allow user download


--------------------------------------------------------------------------------------------

In [28]:
# Load prompts

router_plan_sys = load_prompt_template("../prompts/router/router-plan-sys.md")
router_gen_user = load_prompt_template("../prompts/router/router-gen-user.md")

css_variables = load_prompt_template("../prompts/css/variables_small.md")
css_selectors = load_prompt_template("../prompts/css/selectors.md")

output_path = "../output"

In [29]:
# Project Plan
framework = "React"
language = "JS"
project_plan_sys = load_prompt_template("../prompts/project/project-plan-sys.md")
project_plan_user = load_prompt_template("../prompts/project/project-plan-user.md").format(FRAMEWORK=framework, LANGUAGE=language, USER_REQUIREMENT="Create a modern online bookstore that allows users to browse, search books.")
project_plan_user

'Create a detailed project plan for a website based on the specified requirements:\n\nFramework: React\nLanguage: JS\nProject Requirements: Create a modern online bookstore that allows users to browse, search books.'

In [30]:
project_plan_json = call_llm(user_instruction=project_plan_user, system_instruction=project_plan_sys)
write_json_str(project_plan_json, f"{output_path}/project/project-plan.json")

In [31]:
# Page Generation
page_plan = read_json(f"{output_path}/project/project-plan-react-js-multi.json")
page_gen_common_sys = load_prompt_template("../prompts/page/gen/common.md")
page_gen_react_sys = load_prompt_template("../prompts/page/gen/react-only.md")
page_gen_sys_prompt = page_gen_common_sys + page_gen_react_sys
page_gen_user = load_prompt_template("../prompts/page/gen/user.md")

print(page_gen_sys_prompt)

for page in page_plan["pages"]:
    user_prompt = page_gen_user.format(CSS_VARIABLES=css_variables, CSS_SELECTORS=css_selectors, PROJECT_PLAN=json.dumps(page_plan), PAGE_NAME=page["name"])
    page_md_code = call_llm(user_instruction=user_prompt, system_instruction=page_gen_sys_prompt)
    write_file(page_md_code, f"{output_path}/{page['filepath']}")

Your are a senior frontend engineer. Generate a single Page component that is production-ready, minimal and uses the project's custom CSS library (see rules below).
The project was scaffolded with Vite; do not use Node-only globals or APIs.

## Inputs (provided by user)
- `project_plan`: an object that includes the framework, language, brief summary of the site and all the pages.
- `page_name`: the target page to generate. The page's full description must be looked up from `project_plan.pages` by name.
(Use only information found in `project_plan` for this page unlcess otherwise stated.)

## Custom CSS rules
- Prefer the provided custom CSS utilities/classes in the project
- If the page requires styles not covered by the custom CSS:
    - You my define miniimal additional CSS classes yourself, but they must live inside the page file.
    - Do not create or import separate CSS files.
    - Do not use inline `style={...}` / `style="..."` except for truly one-off accessibility fixes.
    